In [2]:
# from ultralytics import YOLO
# import cv2

# # =======================
# # 1. Load model YOLO
# # =======================
# model = YOLO(r"C:\Users\Nico Marvels\EagleEyes\data windu\video\frame - Original\yolo_dataset\runs\detect\my_yolo_training\weights\best.pt")  # Ganti dengan model .pt kamu

# # =======================
# # 2. Buka video
# # =======================
# video_path = r"C:\Rekaman CCTV\JPO_TJB\vid1.mov"  # Ganti path video kamu
# cap = cv2.VideoCapture(video_path)

# if not cap.isOpened():
#     print("Gagal membuka video!")
#     exit()

# # Info video asli
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fps = cap.get(cv2.CAP_PROP_FPS)

# # =======================
# # 3. Output video
# # =======================
# output_path = "hasil_deteksi.mp4"
# #fourcc = cv2.VideoWriter_fourcc(*"mp4v")
# #out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# # =======================
# # 4. Loop frame-by-frame
# # =======================
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Jalankan deteksi
#     results = model(frame, conf=0.5)

#     # Frame hasil deteksi (dengan kotak bounding box)
#     annotated_frame = results[0].plot()

#     # Simpan hasil ke file
#    # out.write(annotated_frame)

#     # Tampilkan di layar
#     cv2.imshow("Deteksi YOLO", annotated_frame)

#     # Tekan 'q' untuk keluar sebelum video selesai
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # =======================
# # 5. Bersihkan resource
# # =======================
# cap.release()
# #out.release()
# cv2.destroyAllWindows()

# print(f"✅ Deteksi selesai. Video tersimpan di {output_path}")


In [1]:
from ultralytics import YOLO
import cv2
import os

# =======================
# 1. Class Names
# =======================
class_names = [
    'car',
    'driver_buckled',
    'driver_unbuckled',
    'driver_unknown',
    'kaca',
    'motor_1_helmet',
    'motor_1_nohelmet',
    'motor_2_helmet',
    'motor_2_nohelmet',
    'motor_more_2',
    'passanger_buckled',
    'passanger_unbuckled',
    'passanger_unknown',
    'plat_nomor'
]

# =======================
# 2. Load Model
# =======================
model = YOLO(r"C:\Users\Nico Marvels\EagleEyes\data windu\video\frame - Original\yolo_dataset\runs\detect\my_yolo_training\weights\best.pt")  # Ganti dengan model .pt kamu

# =======================
# 2. Buka video
# =======================
video_path = r"C:\Rekaman CCTV\JPO_TJB\vid1.mov"  # Ganti path video kamu
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Gagal membuka video!")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)

# =======================
# 4. Siapkan Folder Output
# =======================
output_folder = "output_per_kelas2"
os.makedirs(output_folder, exist_ok=True)

# Simpan writer per kelas
writers = {}

# =======================
# 5. Loop Frame-by-Frame
# =======================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Deteksi objek
    results = model(frame, conf=0.5)

    # Ambil hasil deteksi di frame ini
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])  # index kelas dari model
            if cls_id < len(class_names):
                label = class_names[cls_id]
            else:
                label = f"class_{cls_id}"

            # Koordinat bounding box
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            crop_obj = frame[y1:y2, x1:x2]

            # Skip kalau crop kosong
            if crop_obj.size == 0:
                continue

            # Buat VideoWriter untuk kelas ini jika belum ada
            if label not in writers:
                h, w = crop_obj.shape[:2]
                out_path = os.path.join(output_folder, f"{label}.mp4")
                fourcc = cv2.VideoWriter_fourcc(*"mp4v")
                writers[label] = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

            # Simpan objek crop ke video sesuai label
            writers[label].write(crop_obj)

    # Tampilkan frame dengan bounding box di layar
    annotated_frame = results[0].plot()
    cv2.imshow("YOLO Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# =======================
# 6. Bersihkan Resource
# =======================
cap.release()
for w in writers.values():
    w.release()
cv2.destroyAllWindows()

print(f"✅ Semua objek terdeteksi sudah disimpan di folder '{output_folder}'")



0: 384x640 3 cars, 1 driver_buckled, 3 kacas, 1 motor_1_helmet, 2 plat_nomors, 253.0ms
Speed: 10.5ms preprocess, 253.0ms inference, 4.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 driver_buckled, 1 driver_unbuckled, 3 kacas, 1 motor_1_helmet, 3 plat_nomors, 315.9ms
Speed: 31.3ms preprocess, 315.9ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 driver_buckled, 3 kacas, 1 motor_1_helmet, 3 plat_nomors, 202.0ms
Speed: 37.3ms preprocess, 202.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 driver_buckled, 1 driver_unbuckled, 3 kacas, 2 motor_1_helmets, 3 plat_nomors, 219.2ms
Speed: 6.5ms preprocess, 219.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 driver_unbuckled, 3 kacas, 1 motor_1_helmet, 3 plat_nomors, 182.9ms
Speed: 54.5ms preprocess, 182.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2 d

In [4]:
from ultralytics import YOLO
import cv2
import os

# =======================
# 1. Class Names
# =======================
class_names = [
    'car',
    'driver_buckled',
    'driver_unbuckled',
    'driver_unknown',
    'kaca',
    'motor_1_helmet',
    'motor_1_nohelmet',
    'motor_2_helmet',
    'motor_2_nohelmet',
    'motor_more_2',
    'passanger_buckled',
    'passanger_unbuckled',
    'passanger_unknown',
    'plat_nomor'
]

# =======================
# 2. Load Model
# =======================
model = YOLO(r"C:\Users\Nico Marvels\EagleEyes\data windu\video\frame - Original\yolo_dataset\runs\detect\my_yolo_training\weights\best.pt")  # Ganti dengan model .pt kamu

# =======================
# 2. Buka video
# =======================
video_path = r"C:\Users\Nico Marvels\EagleEyes\static\video\rek2.mp4"  # Ganti path video kamu

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("❌ Gagal membuka video!")
    exit()

# Ambil resolusi & fps
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# =======================
# 4. Siapkan Output Video
# =======================
output_folder = "hasil_deteksi"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "deteksi3.mp4")

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# =======================
# 5. Loop Frame-by-Frame
# =======================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Deteksi objek di frame
    results = model(frame, conf=0.5)

    # Ganti label default YOLO dengan class_names custom
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            if cls_id < len(class_names):
                r.names[cls_id] = class_names[cls_id]
            else:
                r.names[cls_id] = f"class_{cls_id}"

    # Buat frame dengan bounding box
    annotated_frame = results[0].plot()

    # Simpan ke file output
    out.write(annotated_frame)

    # Tampilkan di layar
    cv2.imshow("YOLO Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# =======================
# 6. Bersihkan Resource
# =======================
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Video hasil deteksi tersimpan di: {output_path}")



0: 384x640 1 car, 1 driver_buckled, 1 kaca, 2 motor_2_helmets, 2 plat_nomors, 156.5ms
Speed: 5.4ms preprocess, 156.5ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 driver_buckled, 1 kaca, 2 motor_2_helmets, 4 plat_nomors, 146.2ms
Speed: 6.3ms preprocess, 146.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 driver_buckled, 1 kaca, 2 motor_2_helmets, 3 plat_nomors, 133.0ms
Speed: 6.4ms preprocess, 133.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 driver_buckled, 1 kaca, 2 motor_2_helmets, 1 plat_nomor, 132.5ms
Speed: 5.4ms preprocess, 132.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 driver_buckled, 1 kaca, 1 plat_nomor, 125.9ms
Speed: 4.4ms preprocess, 125.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 driver_buckled, 1 kaca, 1 plat_nomor, 123.7ms
Speed: 5.0ms preprocess, 123

In [1]:
from ultralytics import YOLO
import cv2
import os

# =======================
# 1. Class Names
# =======================
class_names = [
    'car',
    'driver_buckled',
    'driver_unbuckled',
    'driver_unknown',
    'kaca',
    'motor_1_helmet',
    'motor_1_nohelmet',
    'motor_2_helmet',
    'motor_2_nohelmet',
    'motor_more_2',
    'passanger_buckled',
    'passanger_unbuckled',
    'passanger_unknown',
    'plat_nomor'
]

# =======================
# 2. Load Model
# =======================
model = YOLO(r"C:\Users\Nico Marvels\EagleEyes\data windu\video\frame - Original\yolo_dataset\runs\detect\my_yolo_training\weights\best.pt")  # Ganti dengan model .pt kamu

# =======================
# 2. Buka video
# =======================
video_path = r"C:\Rekaman CCTV\JPO_TJB\vid1.mov"  # Ganti path video kamu
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Gagal membuka video!")
    exit()

# =======================
# 4. Siapkan Folder Output
# =======================
output_folder = "output_per_kelas"
os.makedirs(output_folder, exist_ok=True)

# Hitung jumlah gambar per kelas
counter = {name: 0 for name in class_names}

# =======================
# 5. Loop Frame-by-Frame
# =======================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Deteksi objek
    results = model(frame, conf=0.5)

    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            if cls_id < len(class_names):
                label = class_names[cls_id]
            else:
                label = f"class_{cls_id}"

            # Koordinat bounding box
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            crop_obj = frame[y1:y2, x1:x2]

            # Skip kalau crop kosong
            if crop_obj.size == 0:
                continue

            # Buat folder kelas kalau belum ada
            class_folder = os.path.join(output_folder, label)
            os.makedirs(class_folder, exist_ok=True)

            # Simpan sebagai JPG
            counter[label] += 1
            filename = os.path.join(class_folder, f"{label}_{counter[label]}.jpg")
            cv2.imwrite(filename, crop_obj)

    # Tampilkan video dengan bounding box
    annotated_frame = results[0].plot()
    cv2.imshow("YOLO Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# =======================
# 6. Bersihkan Resource
# =======================
cap.release()
cv2.destroyAllWindows()

print(f"✅ Semua objek tersimpan di folder '{output_folder}' sesuai kelasnya")



0: 384x640 3 cars, 1 driver_buckled, 3 kacas, 1 motor_1_helmet, 2 plat_nomors, 229.2ms
Speed: 12.5ms preprocess, 229.2ms inference, 19.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 driver_buckled, 1 driver_unbuckled, 3 kacas, 1 motor_1_helmet, 3 plat_nomors, 123.4ms
Speed: 17.3ms preprocess, 123.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 driver_buckled, 3 kacas, 1 motor_1_helmet, 3 plat_nomors, 157.5ms
Speed: 5.2ms preprocess, 157.5ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 driver_buckled, 1 driver_unbuckled, 3 kacas, 2 motor_1_helmets, 3 plat_nomors, 121.2ms
Speed: 5.5ms preprocess, 121.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 driver_unbuckled, 3 kacas, 1 motor_1_helmet, 3 plat_nomors, 126.8ms
Speed: 5.6ms preprocess, 126.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2 dr